In [1]:
import pandas as pd

df = pd.read_csv(filepath_or_buffer='dm2_dataset_2425_imdb/preprocessed_full.csv')
df

,Unnamed: 0,originalTitle,rating,startYear,endYear,runtimeMinutes,awardWins,numVotes,worstRating,bestRating,...,genres,castNumber,companiesNumber,averageRating,regions,externalLinks,writerCredits,directorsCredits,soundMixes,quotesTotal
0,0,Carmencita,"(5, 6]",1894,1894.0,1.0,0,2089,1,10,...,"Documentary,Short",1,3,5.7,"['DE', 'US', 'HU', 'GR', 'RU', 'UA', 'JP']",9,0,1,['silent'],0
1,1,Un bon bock,"(5, 6]",1892,1892.0,12.0,0,183,1,10,...,"Animation,Short",0,0,5.4,"['HU', 'DE', 'FR', 'RO', 'RU', 'JP']",3,0,1,['silent'],0
2,2,Chinese Opium Den,"(4, 5]",1894,1894.0,1.0,0,195,1,10,...,Short,0,1,5.0,"['US', 'DE', 'PT', 'HU', 'RU']",4,0,1,['silent'],0
3,3,Edison Kinetoscopic Record of a Sneeze,"(5, 6]",1894,1894.0,1.0,1,2237,1,10,...,"Documentary,Short",1,6,5.4,"['UA', 'JP', 'HU', 'US', 'DE', 'RU']",6,0,1,['silent'],0
4,4,L'arrivée d'un train à La Ciotat,"(7, 8]",1896,1896.0,1.0,0,13115,1,10,...,"Documentary,Short",6,5,7.4,"['HU', 'DE', 'TR', 'FI', 'FR', 'IT', 'ES', 'SK...",21,0,2,['silent'],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149526,149526,Nuestra película,"(6, 7]",1993,1993.0,96.0,0,11,1,10,...,Documentary,0,2,7.0,['CO'],1,1,1,[],0
149527,149527,Eco,"(6, 7]",2019,2019.0,14.0,0,15,1,10,...,"Drama,Short",6,3,6.4,['ES'],5,1,1,[],0
149528,149528,Women Take Center Stage,"(5, 6]",2019,2019.0,NaN,0,12,1,10,...,Reality-TV,1,0,5.6,[],0,0,0,[],0
149529,149529,Horrid Henry and the Christening Crisis,"(7, 8]",2011,2011.0,10.0,0,9,1,10,...,"Adventure,Animation,Comedy",7,0,7.1,[],0,3,1,[],0


In [3]:
df.columns

Index(['Unnamed: 0', 'originalTitle', 'rating', 'startYear', 'endYear',
       'runtimeMinutes', 'awardWins', 'numVotes', 'worstRating', 'bestRating',
       'totalImages', 'totalVideos', 'totalCredits', 'criticReviewsTotal',
       'titleType', 'awardNominationsExcludeWins', 'canHaveEpisodes',
       'isRatable', 'isAdult', 'numRegions', 'userReviewsTotal', 'ratingCount',
       'countryOfOrigin', 'genres', 'castNumber', 'companiesNumber',
       'averageRating', 'regions', 'externalLinks', 'writerCredits',
       'directorsCredits', 'soundMixes', 'quotesTotal'],
      dtype='object')

In [ ]:
from support import convert_string_list

